In [11]:
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader
from tqdm import tqdm  # For nice progress bar!
import torchvision

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load the Data

In [7]:
batch_size = 16
trans_train =  transforms.Compose([transforms.Resize(75),
                                       transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(75),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])

trans_valid = transforms.Compose([transforms.Resize(75),
                                      transforms.CenterCrop(75),
                                      transforms.ToTensor()])

data_dir = '/content/drive/MyDrive/Kaggle Datasets/Cat Dog'

train_dataset = datasets.ImageFolder(data_dir + '/train',
                                  transform=trans_train)
test_dataset = datasets.ImageFolder(data_dir + '/test',
                                 transform=trans_valid)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


# Hyperparameters

In [10]:
num_classes = 2
learning_rate = 1e-3

num_epochs = 5

# Load pretrain model & modify it

In [17]:
model = torchvision.models.vgg16(weights="DEFAULT")

If you want to do finetuning then set requires_grad = `True`
Remove these two lines if you want to train entire model,
and only want to load the pretrain weights.

In [18]:
## Freezing the weights
for param in model.parameters():
    param.requires_grad = False

model.avgpool = nn.Identity()
model.classifier = nn.Sequential(
    nn.Linear(2048, 100), nn.ReLU(), nn.Linear(100, 1)
)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Loss and optimizer

In [19]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network

In [30]:
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores.squeeze(), targets.float())

        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")



100%|██████████| 35/35 [03:49<00:00,  6.56s/it]


Cost at epoch 0 is 0.57623


100%|██████████| 35/35 [00:50<00:00,  1.43s/it]


Cost at epoch 1 is 0.51508


100%|██████████| 35/35 [00:47<00:00,  1.37s/it]


Cost at epoch 2 is 0.48268


100%|██████████| 35/35 [00:49<00:00,  1.41s/it]


Cost at epoch 3 is 0.46817


100%|██████████| 35/35 [00:48<00:00,  1.38s/it]

Cost at epoch 4 is 0.43153


# Check accuracy on training & test to see how good our model


In [32]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples




check_accuracy(train_loader, model)


tensor(0.5027)

In [33]:
check_accuracy(test_loader, model)

tensor(0.4812)